<h1 align="center">
 <b>PROYECTO INDIVIDUAL Nº2</b>
</h1>

# <h1 align="center">**`Siniestros viales`**</h1>
<h1 align="center" style="color: yellow;">Londero Walter Oscar</h1>

<p align="center">
<img src = "https://static.lajornadaestadodemexico.com/wp-content/uploads/2022/08/Siniestros-viales.jpg" height = 500>
<p>

# Proyecto Individual Nº2 - Siniestros Viales

## **Descripción del Problema**

Los siniestros viales son eventos que involucran vehículos en las vías públicas y pueden tener graves consecuencias. En Buenos Aires, estos incidentes son una preocupación debido al alto volumen de tráfico y densidad poblacional. Reducir las tasas de mortalidad por siniestros viales es crucial para mejorar la seguridad vial. En Argentina, los siniestros viales causan alrededor de 4.000 muertes al año, siendo una de las principales causas de muertes violentas. El Observatorio de Movilidad y Seguridad Vial solicita un proyecto de análisis de datos para ayudar a reducir las víctimas fatales de siniestros viales en Buenos Aires, proporcionando un dataset sobre homicidios en siniestros viales ocurridos entre 2016-2021.

El dataset se encuentra en: https://data.buenosaires.gob.ar/dataset/victimas-siniestros-viales

### Contexto

En Argentina, los siniestros viales causan un alto número de muertes cada año, superando incluso a otras causas de muerte violenta. El Observatorio de Movilidad y Seguridad Vial de Buenos Aires busca reducir estas cifras proporcionando información para tomar medidas efectivas.

### Rol a Desarrollar

El proyecto implica el análisis de datos sobre homicidios en siniestros viales en Buenos Aires entre 2016 y 2021. El objetivo es proporcionar información que permita a las autoridades locales tomar medidas para reducir la cantidad de víctimas fatales.


# <span style="color: #FF9843">**ETL**</span>

## <span style="color: #FFDD95">**Extracción**</span>

_Estas son las Librerias necesarias para la ejecución del Proyecto:_

In [ ]:
!pip freeze > requirements.txt

In [ ]:
with open('requirements.txt', 'r') as archivo:
    contenido = archivo.read()
    print(contenido)

In [ ]:
import pandas as pd
import numpy as np
from summarytools import dfSummary
import requests


In [ ]:
# URL de exportación de Google Sheets
url = 'https://docs.google.com/spreadsheets/d/1nq00jGIZHQ1RLSET43zKnUsMsoFb-pBg/export?format=xlsx'

# Nombre del archivo donde se guardará la descarga
filename = 'spreadsheet.xlsx'

# Hacer la solicitud GET a la URL
response = requests.get(url)

# Guardar el contenido de la respuesta en un archivo
with open(filename, 'wb') as file:
    file.write(response.content)

print(f'El archivo se ha descargado y guardado como {filename}')

## <span style="color: #FFDD95">**Transformación**</span>

### <span style="color: #8DECB4">**1-Explorar y Limpiar Datos**</span>

****
<span style="color: #FAEF5D">Carga de los datos:</span>

Se importa la información de cada hoja del archivo en un df diferente:

In [ ]:
# Read the downloaded spreadsheet into DataFrames
df_hechos = pd.read_excel(filename, sheet_name='HECHOS')
df_victimas = pd.read_excel(filename, sheet_name='VICTIMAS')

Hacemos un primer análisis de la información:

In [ ]:
# Print DataFrames
print("Datos de la hoja HECHOS:")
print(df_hechos)

In [ ]:
print("\nDatos de la hoja VICTIMAS:")
print(df_victimas)

Obtenemos la información general de los df y analizamos tipos de datos y futuras transformaciones:

In [ ]:
# Obtener información general del DataFrame
df_hechos.info()

In [ ]:
df_victimas.info()

Como el reemplazante "SD" de datos nulos posee diferencias con respecto a mayúsculas y minúsculas debemos unificarlos:

In [ ]:
df_hechos.isnull().sum()

Los valores nulos son significativos en el df_hechos en dos columnas principalmente, Altura con 567 valores faltantes y Cruce con 171 faltantes. Se van a conservar las columnas pero se tendra en cuanta para analisis posteriores que puedan involucrar estas columnas.

Reemplazar los valores nulos en HORA y HH con 25, debido a que es un tipo numérico y facilmente reconocible posteriormente en el análisisya ya que no corresponde a una hora válida:

In [ ]:
# Reemplazar los valores nulos en las columnas 'HORA' y 'HH' con 0
df_hechos['HORA'] = df_hechos['HORA'].fillna('25:00')
df_hechos['HH'] = df_hechos['HH'].fillna(25)

Reemplazar los valores nulos en LUGAR_DEL_HECHO, Calle, Altura, Cruce, Direccion Normalizada, VICTIMA, y ACUSADO con 'SD'.


In [ ]:
# Reemplazar los valores nulos en las columnas especificadas con 'SD'
columns_to_replace_with_sd = ['LUGAR_DEL_HECHO', 'Calle', 'Altura', 'Cruce', 'Dirección Normalizada', 'VICTIMA', 'ACUSADO']
df_hechos[columns_to_replace_with_sd] = df_hechos[columns_to_replace_with_sd].fillna('SD')

In [ ]:
df_victimas.isnull().sum()

El df_victimas no contiene datos nulos.

Análisis de columnas con valores NaN

In [ ]:
df_hechos[["Calle", "Altura", "Dirección Normalizada", "Cruce"]][:5]

Corroboramos si podemos extraer de la dirección normalizada los datos necesarios para completar la columna Altura y esta información no se encuentra en ningu otro lugar. Por lo que el análisis debe continuar sin estos datos.

In [ ]:
# Definir las columnas a revisar
columns_to_check = ['Calle', 'Altura', 'Dirección Normalizada', 'Cruce']

# Contar la cantidad de valores 'SD' en cada columna
sd_counts = df_hechos[columns_to_check].apply(lambda x: (x == 'SD').sum())

# Mostrar los resultados
print(sd_counts)

La columna **`Altura`** posee gran cantidad de datos faltantes y con la informacion que se tine no hay forma de completarla. En este caso la cantidad de faltantes corresponde a más del 80% de los datos. En este caso debemos eliminarla:

In [ ]:
# Se elimina la columna "Altura"
df_hechos= df_hechos.drop("Altura", axis=1)
df_hechos.columns

Ahora podemos ver si hay información duplicada en nuestro df:

In [ ]:
df_hechos.duplicated().sum()

In [ ]:
df_victimas.duplicated().sum()

En los dataframes no hubo entradas repetidas y el shape de los archivos sigue el mismo:

In [ ]:
num_rows = df_hechos.shape[0]
print("Número de filas en df_hechos:", num_rows)

In [ ]:
num_rows = df_victimas.shape[0]
print("Número de filas en df_victimas:", num_rows)

Corroboramos la info de elementos nulos en df_victimas:

In [ ]:
df_victimas

In [ ]:
df_victimas.isnull().sum()

Renombrar la columna: Se usa el método rename con el parámetro inplace=True para cambiar el nombre de la columna ID_hecho a ID.

In [ ]:
# Renombrar la columna ID_hecho a ID
df_victimas.rename(columns={'ID_hecho': 'ID'}, inplace=True)

    Esto será útil para hacer posteriormente el Join.

Verificamos que todo esta completo y no tenemos valores nulos en Nuestros DataFrames:

In [ ]:
df_hechos.isnull().sum()

In [ ]:
# Verificar el cambio
df_victimas.isnull().sum()

### <span style="color: #8DECB4">**2-Explorar y Transformar Datos**</span>

****
<span style="color: #FAEF5D">Correccón de Mayusculas y Minusculas:</span>

En este apartado se ha realizado la transformación de manera correcta sin ningún inconveniente, la unificación de la información a mayúsculas ayuda a la prolijidad y a evitar errores de escritura en formulas o funciones, evitando así chequear en todo momento el nombre de columnas para no escribir mal.

In [ ]:
def convertir_columnas(df):
  """
  Convierte los nombres de las columnas de un DataFrame a mayúsculas y elimina los espacios.

  Parámetros:
    df: El DataFrame al que se le aplicarán las modificaciones.

  Retorno:
    Un DataFrame con los nombres de las columnas modificados.
  """
  df.columns = df.columns.str.upper().str.replace(" ", "_")
  return df

# Aplicamos la función a cada DataFrame
df_hechos = convertir_columnas(df_hechos)
df_victimas = convertir_columnas(df_victimas)

Hacemos una presentacón de como esta presentada la informacion en nuestros DataFrames:

In [ ]:
df_hechos.head(2)

In [ ]:
df_victimas.head(2)

In [ ]:
dfSummary(df_hechos)

In [ ]:
dfSummary(df_victimas)

### <span style="color: #8DECB4">**3-Unificar DataSet**</span>

Los datasets comparten información repetida por lo que debemos eliminar en alguno de ellos estas columnas que se repetirán en el df final:

In [ ]:
# Se eliminan las columnas repetidas
df_victimas = df_victimas.drop(['FECHA', 'AAAA', 'MM', 'DD', 'VICTIMA'], axis=1)
df_victimas.columns

In [ ]:
df_victimas

In [ ]:
df_hechos

In [ ]:
len(df_hechos)

In [ ]:
len(df_victimas)

Para fusionar los df se utiliza la función Merge.

In [ ]:
viales_clean = df_victimas.merge(df_hechos, on="ID", how="left")
viales_clean.columns

In [ ]:
viales_clean

## <span style="color: #FFDD95">**Carga**</span>

En cierta parte de nuestro dataset tenemos un salto de pagina que genera un error en el guardado del csv, por lo que con el código siguiente eliminamos el espacio y el dataset se guarda correctamente.

In [ ]:
# Eliminar saltos de línea en todas las columnas
for col in viales_clean.columns:
    viales_clean[col] = viales_clean[col].astype(str).str.replace('\n', ' ').str.replace('\r', ' ')

Como hay errores en la columna MM la pasamos a entero para luego usarla en la formula de creación de la columna Semestre:

In [ ]:
# Convertir la columna 'MM' a tipo numérico si es necesario
viales_clean['MM'] = pd.to_numeric(viales_clean['MM'], errors='coerce')

# Crear la columna 'Semestre' basada en el mes
viales_clean['Semestre'] = viales_clean['MM'].apply(lambda x: '1' if x <= 6 else '2')

viales_clean

In [ ]:
# Archivos DataFrame a exportar
viales_clean.to_csv('viales_clean.csv', index=False)